# 🤖 AgentPro Interview Task Generator

This notebook demonstrates how to use AgentPro to generate interview tasks and create presentation slides based on job descriptions and candidate details.

### Features:
- Generate interview tasks based on job description
- Create presentation slides with tasks
- Customize tasks based on candidate persona
- Export tasks as PowerPoint presentation

## Step 1: Install Dependencies

First, let's install the required packages.

In [ ]:
!pip install gradio python-pptx

## Step 2: Set Your API Keys

Set your OpenAI API key to use the language model.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

## Step 3: Create the Interview Task Generator

Let's create a function that generates interview tasks and creates slides.

In [ ]:
from agentpro import AgentPro
from agentpro.tools.slide_tool import SlideGenerationTool
import gradio as gr
import json

# Initialize the agent with slide generation tool
agent = AgentPro(tools=[SlideGenerationTool()])

def generate_interview_tasks(job_description, candidate_persona):
    # Generate tasks using the agent
    prompt = f"""
    Based on the following job description and candidate persona, generate 5 interview tasks.
    Format each task as a slide with a title and detailed content.
    Make the tasks challenging but fair, and relevant to the role.
    
    Job Description:
    {job_description}
    
    Candidate Persona:
    {candidate_persona}
    
    Return the tasks in JSON format with the following structure:
    [
        {{"slide_title": "Task 1 Title", "content": "Task 1 detailed description"}},
        {{"slide_title": "Task 2 Title", "content": "Task 2 detailed description"}},
        ...
    ]
    """
    
    # Get response from agent
    response = agent(prompt)
    
    # Try to parse the response as JSON
    try:
        tasks = json.loads(response)
        # Generate slides
        slide_tool = SlideGenerationTool()
        result = slide_tool.run(tasks)
        return result
    except json.JSONDecodeError:
        return "Error: Could not generate tasks in the correct format."

# Create Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("## 🤖 Interview Task Generator")
    gr.Markdown("Generate interview tasks and create presentation slides based on job description and candidate details.")
    
    with gr.Row():
        with gr.Column():
            job_desc = gr.Textbox(
                label="Job Description",
                placeholder="Enter the job description here...",
                lines=10
            )
            candidate_info = gr.Textbox(
                label="Candidate Persona",
                placeholder="Enter candidate details (experience, skills, background)...",
                lines=5
            )
            generate_btn = gr.Button("Generate Tasks")
        
        with gr.Column():
            output = gr.Textbox(label="Output")
    
    # Example inputs
    gr.Examples(
        examples=[
            [
                "Software Engineer position requiring 3+ years of experience in Python, Django, and React. The role involves developing web applications, implementing REST APIs, and working with databases.",
                "Candidate has 4 years of experience in web development, strong Python skills, and experience with Django and React. They have worked on e-commerce platforms and have a computer science degree."
            ],
            [
                "Data Scientist position requiring expertise in machine learning, Python, and data analysis. The role involves building predictive models, analyzing large datasets, and presenting insights to stakeholders.",
                "Candidate has 5 years of experience in data science, strong Python and SQL skills, and experience with machine learning frameworks. They have worked in the healthcare industry and have a master's degree in data science."
            ]
        ],
        inputs=[job_desc, candidate_info],
        outputs=output,
        fn=generate_interview_tasks,
        cache_examples=True
    )
    
    generate_btn.click(
        fn=generate_interview_tasks,
        inputs=[job_desc, candidate_info],
        outputs=output
    )

# Launch the app
app.launch(share=True)